In [1]:
import owlready2 as owl
from owlready2 import *

import numpy as np
import os
from datetime import datetime
import random

owlready2.reasoning.JAVA_MEMORY = 2000

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [2]:
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/goslimyeast.xml.owl'
dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/galennorm.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/gonorm.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/generations.owl'

In [3]:
onto = get_ontology(dir)
onto = onto.load()
gcas = list(onto.general_class_axioms())
classes = list(onto.classes())
roles = list(onto.properties())

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.co-ode.org/ontologies/galen#BodyStructure
  http://www.co-ode.org/ontologies/galen#OrganicSolidStructure
  http://www.co-ode.org/ontologies/galen#SolidStructure
  http://www.co-ode.org/ontologies/galen#PhysicalStructure
  http://www.co-ode.org/ontologies/galen#GeneralisedStructure
  http://www.co-ode.org/ontologies/galen#Phenomenon
  http://www.co-ode.org/ontologies/galen#DomainCategory
  http://www.co-ode.org/ontologies/galen#TopCategory
  http://www.co-ode.org/ontologies/galen#ViralInfectionLesion
  http://www.co-ode.org/ontologies/galen#InfectionLesion
  http://www.co-ode.org/ontologies/galen#PathologicalBodyStructure

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://www.co-ode.org/ontologies/galen#Displacement
  http://www.co-ode.org/ontologies/galen#LinearStructure
  http://www.co-ode.org/ontologies/galen#LiquidBlood
  http://www.co-ode.org/ontologies/ga

In [4]:
class CanonicalModelElements:

    concept_names = {}
    concept_intersections = {}
    concept_restrictions = {}
    all_concepts = {}

    def __init__(self, concept):
        self.concept = concept
        self.name = self.get_name()
        self.get_element_dict()

    def get_name(self):

        # add \Top
        
        if type(self.concept) == ThingClass:
            return self.concept.name

        elif type(self.concept) == Restriction:
            return 'exists_' + self.concept.property.name + '.' + self.concept.value.name
        
        else:
            return 'And_' + ''.join(sorted(self.concept.Classes[0].name + self.concept.Classes[1].name)) # The name is sorted to avoid that (e.g) (A \and B) and (B \and A) are treated as different concepts
        
    def get_element_dict(self):

        if type(self.concept) == ThingClass:
            CanonicalModelElements.concept_names[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == Restriction:
            CanonicalModelElements.concept_restrictions[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == And:
            CanonicalModelElements.concept_intersections[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

In [5]:
'''
Function for preprocessing
concepts to act as input
for the creation of the
canonical model. Stores
concepts in dictionary
form.
'''

def get_canonical_model_elements(concept_names_iter, gcas_iter):

    for concept_name in concept_names_iter:
        CanonicalModelElements(concept_name)
        for construct in concept_name.constructs():
            CanonicalModelElements(construct)

    print('')
    print('')
    print('All concept Names have been preprocessed for the creation of the canonical model!')

    # Preprocessses the right side of SubClassOf inclusions
        
    for concept_name in list(concept_names_iter):
        concept_name_list = concept_name.is_a
        concept_right_side_caninterp = [CanonicalModelElements(concept_name) for concept_name in concept_name_list]

    print('All concepts appearing on the right side of SubClassOf inclusions have been preprocessed for the creation of the canonical model!')

    # Preprocessses the left and right sides of GCAs
    
    for gca in list(gcas_iter):
        gca_left_side = gca.left_side
        gca_left_side_caninterp = CanonicalModelElements(gca_left_side)
        gca_right_side = gca.is_a

        if len(list(gca_right_side)) == 1:
            gcas_right_side_caninterp = CanonicalModelElements(gca_right_side[0])
        else:
            print('Hello mr buggy')
            gcas_right_side_caninterp = [CanonicalModelElements(concept) for concept in gca_right_side]
    
    print('All concepts in both sides of General Class Axioms have been preprocessed for the creation of the canonical model!')

In [6]:
class CanonicalModel:

    concept_canonical_interpretation = {}
    role_canonical_interpretation = {}

    def __init__(self, concept_names_dict, concept_intersections_dict, concept_restrictions_dict, all_concepts_dict, role_names_iter):
        
        self.domain = all_concepts_dict
        self.concept_names_dict = concept_names_dict
        self.concept_restrictions_dict = concept_restrictions_dict
        self.concept_intersections_dict = concept_intersections_dict

        self.role_names_iter = role_names_iter

        self.concept_canonical_interp = self.get_concept_name_caninterp() # These are only used to build the concept_canonical_interpretation and role_canonical_interpretation class attributes
        self.role_canonical_interp = self.get_role_name_caninterp()       # The functions do not return anything, they just update the class variables

    def get_concept_name_caninterp(self):

        # The variable concept is a string containing the name of an element of the domain of the canonical model
        # The key to the concept_names_dict variable corresponds to concept.name
        # This name can be used to access the concept in owlready2's format

        for concept in list(self.concept_names_dict.keys()):

            CanonicalModel.concept_canonical_interpretation[concept] = []
            
            superclasses = self.domain[concept].concept.ancestors(include_self=True, include_constructs=True) # The self.domain[concept] is used to access the CanonicalModelElements type of object,
                                                                                                               # and the attribute .concept is used to access the concept in owlready2 format
                                                                                                              
            for superclass in superclasses:

                if type(superclass) == ThingClass:
                    CanonicalModel.concept_canonical_interpretation[concept].append(superclass.name)

                elif type(superclass) == Restriction:
                    CanonicalModel.concept_canonical_interpretation[concept].append('exists_' + superclass.property.name + '.' + superclass.value.name)

                elif type(superclass) == And:
                    if 'And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)) in CanonicalModel.concept_canonical_interpretation[concept]:
                        pass
                    else:
                        CanonicalModel.concept_canonical_interpretation[concept].append('And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)))

    def get_role_name_caninterp(self):

        # First case from Definition 10

        for role_name in self.role_names_iter:

            role_name = role_name.name # Accesses the property type object's name as a string
            CanonicalModel.role_canonical_interpretation[role_name] = []

            for restriction_name in self.concept_restrictions_dict.keys(): # Where restriction_name denotes a \exists r.B type of concept 'exists_' + self.concept.property.name + '.' + self.concept.value.name
                c_B = self.concept_restrictions_dict[restriction_name].concept.value.name

                if role_name in restriction_name: # This will not work, rewrite
                    superclasses = self.domain[restriction_name].concept.ancestors(include_self=True, include_constructs=False)

                    for superclass in superclasses:
                        super_superclasses = superclass.ancestors(include_self=True, include_constructs=True)

                        for super_superclass in super_superclasses:

                            if type(super_superclass) == ThingClass:
                                c_D = super_superclass.name
                                CanonicalModel.role_canonical_interpretation[role_name].append(tuple(c_D, c_B))

                            elif type(super_superclass) == Restriction:
                                c_D = 'exists_' + super_superclass.property.name + '.' + super_superclass.value.name
                                CanonicalModel.role_canonical_interpretation[role_name].append(tuple(c_D, c_B))

                            elif type(super_superclass) == And:
                                c_D = 'And_' + ''.join(sorted(super_superclass.Classes[0].name + super_superclass.Classes[1].name))
                                CanonicalModel.role_canonical_interpretation[role_name].append(tuple(c_D, c_B))

                else:
                    pass

        # Second case from Definition 10
            
            for role_name in self.role_names_iter:
                pass

In [7]:
def create_canonical_model(onto_dir):

    onto = get_ontology(onto_dir)
    onto = onto.load()
    gcas_iter = onto.general_class_axioms()
    concept_names_iter = onto.classes()
    role_names_iter = list(onto.properties())

    get_canonical_model_elements(concept_names_iter, gcas_iter)

    print('============================================================================')
    print('')
    print('Starting to reason.')
    print('')

    #with onto:
    #    sync_reasoner(infer_property_values=True) # Performs inference in the ontology to derive concept inclusions

    print('============================================================================')
    print('')
    print('Done reasoning. Creating the canonical model.')
    print('')
    canmodel = CanonicalModel(CanonicalModelElements.concept_names, CanonicalModelElements.concept_intersections, CanonicalModelElements.concept_restrictions, CanonicalModelElements.all_concepts, role_names_iter)
    print('============================================================================')
    print('')
    print('Concluded creating canonical model.')

    return canmodel

In [8]:
canmodel = create_canonical_model(dir)



All concept Names have been preprocessed for the creation of the canonical model!
All concepts appearing on the right side of SubClassOf inclusions have been preprocessed for the creation of the canonical model!
All concepts in both sides of General Class Axioms have been preprocessed for the creation of the canonical model!

Starting to reason.


Done reasoning. Creating the canonical model.


Concluded creating canonical model.


In [10]:
concanin = CanonicalModel.concept_canonical_interpretation
rolcanin = CanonicalModel.role_canonical_interpretation
domain = canmodel.domain
restrictions = canmodel.concept_restrictions_dict